# Entrenamiento de películas en Netflix


## Curando el dataset
Para entrenar el modelo usaremos los conjuntos de datos de [IMDB](https://developer.imdb.com/non-commercial-datasets/), title.basics para obtener los títulos, fecha de salida y géneros de las películas y title.crew para los directores de las películas.

Filtramos el conjunto para eliminar todo lo que no sea películas y lo guardamos en un archivo csv más fácil de procesar.


In [ ]:
import pandas as pd

# Cargamos el dataset de IMDB
df = pd.read_csv("datasets/title.basics.tsv", sep ='\t')

# Cogemos solo las películas y guardamos en otro archivo
df = df[df['titleType'] == 'movie']
df.to_csv("datasets/titles_filtered.csv")
df.head(3)

Eliminamos columnas de datos innecesarios y consideramos irrelevantes aquellos registros sin fecha de salida ni género.

In [22]:
import pandas as pd

df = pd.read_csv("datasets/titles_filtered.csv")

# Eliminamos la primera columna, son índices
df = df.drop(df.columns[0], axis=1)

# Eliminamos columnas irrelevantes
df = df.drop(['endYear', 'isAdult', 'titleType'], axis=1)

# Eliminamos películas sin fecha de salida ni géneros
df = df[df['startYear'] != '\\N']
df = df[df['genres'] != '\\N']

# Unificamos los títulos, todos en mayúscula
df['primaryTitle'] = df['primaryTitle'].str.upper()
df['originalTitle'] = df['originalTitle'].str.upper()

# Cambiamos los valores nulos de la duración a 0
df.loc[df['runtimeMinutes'] == '\\N', 'runtimeMinutes'] = 0

# Forzamos el tipo de startYear y runtimeMinutes a ser numérico,
# era mixto por los nulos ('\N')
df['startYear'] = pd.to_numeric(df['startYear'], downcast='integer', errors='coerce')
df['runtimeMinutes'] = pd.to_numeric(df['runtimeMinutes'], downcast='integer', errors='coerce')

df.head(10)

C:\Users\GUILLERMO\AppData\Local\Temp\ipykernel_6724\2247269571.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("datasets/titles_filtered.csv")


,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres
0,tt0000009,MISS JERRY,MISS JERRY,1894,45,Romance
1,tt0000147,THE CORBETT-FITZSIMMONS FIGHT,THE CORBETT-FITZSIMMONS FIGHT,1897,100,"Documentary,News,Sport"
3,tt0000574,THE STORY OF THE KELLY GANG,THE STORY OF THE KELLY GANG,1906,70,"Action,Adventure,Biography"
4,tt0000591,THE PRODIGAL SON,L'ENFANT PRODIGUE,1907,90,Drama
5,tt0000615,ROBBERY UNDER ARMS,ROBBERY UNDER ARMS,1907,0,Drama
6,tt0000630,HAMLET,AMLETO,1908,0,Drama
7,tt0000675,DON QUIJOTE,DON QUIJOTE,1908,0,Drama
8,tt0000679,THE FAIRYLOGUE AND RADIO-PLAYS,THE FAIRYLOGUE AND RADIO-PLAYS,1908,120,"Adventure,Fantasy"
19,tt0000886,"HAMLET, PRINCE OF DENMARK",HAMLET,1910,0,Drama
20,tt0000941,LOCURA DE AMOR,LOCURA DE AMOR,1909,45,Drama


Unimos a nuestro conjunto los directores de cada película a partir del archivo de IMDB.

In [23]:
df_crew = pd.read_csv("datasets/title.crew.tsv", sep ='\t')

df = pd.merge(df, df_crew, on="tconst")

df.head(10)

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,directors,writers
0,tt0000009,MISS JERRY,MISS JERRY,1894,45,Romance,nm0085156,nm0085156
1,tt0000147,THE CORBETT-FITZSIMMONS FIGHT,THE CORBETT-FITZSIMMONS FIGHT,1897,100,"Documentary,News,Sport",nm0714557,\N
2,tt0000574,THE STORY OF THE KELLY GANG,THE STORY OF THE KELLY GANG,1906,70,"Action,Adventure,Biography",nm0846879,nm0846879
3,tt0000591,THE PRODIGAL SON,L'ENFANT PRODIGUE,1907,90,Drama,nm0141150,nm0141150
4,tt0000615,ROBBERY UNDER ARMS,ROBBERY UNDER ARMS,1907,0,Drama,nm0533958,"nm0092809,nm0533958"
5,tt0000630,HAMLET,AMLETO,1908,0,Drama,nm0143333,nm0000636
6,tt0000675,DON QUIJOTE,DON QUIJOTE,1908,0,Drama,nm0194088,nm0148859
7,tt0000679,THE FAIRYLOGUE AND RADIO-PLAYS,THE FAIRYLOGUE AND RADIO-PLAYS,1908,120,"Adventure,Fantasy","nm0091767,nm0877783","nm0000875,nm0877783"
8,tt0000886,"HAMLET, PRINCE OF DENMARK",HAMLET,1910,0,Drama,nm0099901,nm0000636
9,tt0000941,LOCURA DE AMOR,LOCURA DE AMOR,1909,45,Drama,"nm0063413,nm0550220","nm0063413,nm0550220,nm0848502"


Comprobamos la integridad de los tipos de las columnas.

In [24]:
for column in df.columns:
    print(column, ":", pd.api.types.infer_dtype(df[column]))

tconst : string
primaryTitle : string
originalTitle : string
startYear : integer
runtimeMinutes : integer
genres : string
directors : string
writers : string


Guardamos un csv nuevo con el nuevo registro con los datos que usaremos para el entrenamiento.

In [26]:
df.to_csv("datasets/movies_curated.csv")

In [40]:
df = pd.read_csv("datasets/movies_curated.csv")

In [47]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer

#df['genres'] = df['genres'].str.split(',')
#df['directors'] = df['directors'].str.split(',')

mlb_genres = MultiLabelBinarizer(sparse_output=True)
genres_encoded = mlb_genres.fit_transform(df['genres'])

mlb_directors = MultiLabelBinarizer(sparse_output=True)
directors_encoded = mlb_directors.fit_transform(df['directors'])

genres_encoded

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 856258 stored elements and shape (548451, 28)>